In [1]:
import tensorflow as tf

/home/lwh/anaconda3/envs/tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/lwh/anaconda3/envs/tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [5]:
def model_fn_AlexNet(features,labels,mode,params):
    keep_prob = 1
    #定义Alexnet网络
    net = tf.feature_column.input_layer(features, params['feature_columns'])
    #1
    net = tf.layers.conv2d(net,filters = 64,kernel_size = (11,11),strides=(4, 4),padding='valid',activation='relu',name = 'conv1')
    net = tf.layers.max_pooling2d(net,pool_size = (3,3),strides = (2,2),padding='valid',name='pool1')
    
    #3
    net = tf.layers.conv2d(net,filters = 192,kernel_size = (5,5),strides=(1, 1),padding='valid',activation='relu',name = 'conv2')
    net = tf.layers.max_pooling2d(net,pool_size = (3,3),strides = (2,2),padding='valid',name='pool2')
    
    #3
    net = tf.layers.conv2d(net,filters = 384,kernel_size = (3,3),strides=(1, 1),padding='valid',activation='relu',name = 'conv3')
    net = tf.layers.conv2d(net,filters = 384,kernel_size = (3,3),strides=(1, 1),padding='valid',activation='relu',name = 'conv4')
    net = tf.layers.conv2d(net,filters = 256,kernel_size = (3,3),strides=(1, 1),padding='valid',activation='relu',name = 'conv5')
    net = tf.layers.max_pooling2d(net,pool_size = (3,3),strides = (2,2),padding='valid',name='pool5')
    
     # 6th Layer: Flatten -> FC (w ReLu) -> Dropout
    net = tf.reshape(net, [-1, 6*6*256])
    net = tf.layers.Dense(inputs = net,units = 4096,name='fc6')
    net = tf.layers.dropout(net,keep_prob)

    # 7th Layer: FC (w ReLu) -> Dropout
    net = tf.layers.Dense(inputs = net,units = 4096,name='fc7')
    net = tf.layers.dropout(net,keep_prob)

    # 8th Layer: FC and return unscaled activations
    logits = tf.layers.Dense(inputs = net,units = 8,name='fc8')
    
    #predict
    predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)
    
    #损失函数
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    #eval
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)
    
    #train
    optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)       

In [8]:
input_files=['/home/lwh/图片/CK+/GenImgs/train.tfrecords']
dataset=tf.data.TFRecordDataset(input_files)

def parser(record):
    features=tf.parse_single_example(
            record,
            features={
                'label': tf.FixedLenFeature([], tf.float32),
                'img_raw' : tf.FixedLenFeature([], tf.float32),
                }
            )
    img = tf.cast(features['img_raw'], tf.float32)
    return img,features["label"]
dataset=dataset.map(parser)#接受的参数是一个函数

def train_input_fn(features, labels, batch_size):
    iterator=dataset.make_one_shot_iterator()
    #img,label=iterator.get_next()
    return iterator.get_next()  

In [10]:
FERclassifier = tf.estimator.Estimator(model_fn=model_fn_AlexNet)
# Train the Model.
FERclassifier.train(input_fn=train_input_fn,steps=5)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': None, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_train_distribute': None, '_global_id_in_cluster': 0, '_model_dir': '/tmp/tmp1yles7qk', '_task_type': 'worker', '_num_ps_replicas': 0, '_service': None, '_device_fn': None, '_session_config': None, '_keep_checkpoint_max': 5, '_is_chief': True, '_evaluation_master': '', '_log_step_count_steps': 100, '_save_summary_steps': 100, '_master': '', '_task_id': 0, '_save_checkpoints_secs': 600, '_num_worker_replicas': 1, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6f99d09208>}


TypeError: train_input_fn() missing 3 required positional arguments: 'features', 'labels', and 'batch_size'

In [2]:
#
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the read end of the pipeline.
    return dataset.make_one_shot_iterator().get_next()

def my_model_fn(features,labels,mode,params):
    """
   features, # This is batch_features from input_fn
   labels,   # This is batch_labels from input_fn
   mode,     # An instance of tf.estimator.ModeKeys
   params    # Additional configuration
   """
    #定义网络结构
    net = tf.feature_column.input_layer(features, params['feature_columns'])
    for units in params['hidden_units']:
        net = tf.layers.dense(net, units=units, activation=tf.nn.relu)
    logits = tf.layers.dense(net, params['n_classes'], activation=None)
    
    #predict
    predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)
    
    #损失函数
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    #eval
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)
    
    #train
    optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)    

In [3]:
#创立feature_columns
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

classifier = tf.estimator.Estimator(
        model_fn=my_model,
        params={
            'feature_columns': my_feature_columns,
            # Two hidden layers of 10 nodes each.
            'hidden_units': [10, 10],
            # The model must choose between 3 classes.
            'n_classes': 3,
        })
# Train the Model.
classifier.train(
    input_fn=lambda:iris_data.train_input_fn(train_x, train_y, args.batch_size),
    steps=args.train_steps)

NameError: name 'train_x' is not defined